In [1]:
from regain.datasets.hmm import generate_hmm
from regain.hmm.hmm_graphical_lasso import HMM_GraphicalLasso 
import numpy as np
from importlib import reload
import regain.hmm.utils; reload(regain.hmm.utils)
from regain.hmm.utils import results_recap
from regain.hmm.utils import cross_validation#, cross_validation_auto
import pickle as pkl
from sklearn import cluster
from sklearn.mixture import GaussianMixture
from pomegranate import *


/Users/federicociech/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# list of parameters to generate different kind of datasets

states = [2,5,10,15]
dims = [10, 15, 20, 30]
mode_prec = ['regain','regain_random']
mode_mea = ['Normal']#, 'Uniform']

res_all = {}

In [3]:
from sklearn.covariance import empirical_covariance

def compute_precision(X,K,clus):
    
    precisions = []
    N, d = X.shape
    means = np.zeros((K, d))
    
    for i, l in enumerate(np.unique(clus)):
        
        means[i, :] = np.mean(X[np.where(clus == l)[0], :], axis=0)
        emp_cov = empirical_covariance(X - means[i, :],assume_centered=True)
        precisions.append(np.linalg.pinv(emp_cov))
    
    return precisions
    

In [4]:
for st in states:
    for dim in dims:
        for mm in mode_prec:


            print('Case: N_states:',st,',TS dim:',dim,',Prec mode:',mm)

            # Generate a dataset
            dataset = generate_hmm(n_samples=1000,
                                   n_states=st,
                                   n_dim_obs=dim,
                                   mode_precisions=mm,
                                   mode_mean = 'Normal',
                                   transition_type='fixed',
                                   sigma=1)
            # CV hmm_gmm 

            hmm_gmm = HMM_GraphicalLasso(alpha=1,
                                         max_iter = 1000,
                                         n_clusters=st,
                                         verbose=False,
                                         mode='scaled',
                                         warm_restart=True,
                                         repetitions=50,
                                         n_jobs=-1)


            if st == 2:
                alpha_list = np.linspace(35,115 ,20)
            elif st == 5:
                alpha_list = np.linspace(25,55 ,20)
            elif st == 10:
                alpha_list = np.linspace(15,45 ,20)
            elif st == 15:
                alpha_list = np.linspace(5,35 ,20)


            res = cross_validation(hmm_gmm,
                   dataset['data'],
                   params={'alpha':alpha_list,
                           'n_clusters':[st]},
                   n_repetitions=1)

            # define three different models

            hmm_gmm = HMM_GraphicalLasso(alpha=res[0][0], n_clusters=st, verbose=False, mode='scaled',
                                     warm_restart=True, repetitions=50, n_jobs=-1)
            
            #hmm
            
            hmm = HMM_GraphicalLasso(alpha=0, n_clusters=st, verbose=False, mode='scaled',
                                     warm_restart=True, repetitions=50, n_jobs=-1)

            
            # gmm 
            gmm = GaussianMixture(n_components=st)
            #hmm 
            #hmm = HiddenMarkovModel.from_samples(NormalDistribution,
            #                                       n_components= st, X=dataset['data'])
            # spectral clustering 
            
            spectral = cluster.SpectralClustering(n_clusters=st,assign_labels="discretize")
            
            # Kmeans
            kmeans = cluster.KMeans(n_clusters=st)
            

            # fit and save results
            try :
                hmm_gmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'HMM_GMM')] = results_recap(dataset['states'], 
                                                                   hmm_gmm.labels_,
                                                                   dataset['thetas'], 
                                                                   hmm_gmm.precisions_)
            except:
                print("HMM-GMM failed")

            
            try :
                hmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'HMM_GMM')] = results_recap(dataset['states'], 
                                                                   hmm.labels_,
                                                                   dataset['thetas'], 
                                                                   hmm.precisions_)
            except:
                print("HMM failed")
            
            try :
                gmm.fit(dataset['data'])
                res_all[(st, dim, mm, 'GMM')] = results_recap(dataset['states'], 
                                                              gmm.predict(dataset['data']),
                                                              dataset['thetas'], 
                                                              gmm.precisions_)
            except:
                print("GMM failed")
            #try :
            #    hmm.fit(dataset['data'],algorithm='baum-welch')
            #    res_all[(st, dim, mm, 'HMM')] = results_recap(dataset['states'], 
            #                                                      hmm.predict(dataset['data']))
            #except:
            #    print("HMM failed")
            
            try :
                spectral.fit(dataset['data'])
                res_all[(st, dim, mm, 'spectral')] = results_recap(dataset['states'], 
                                                                   spectral.labels_,
                                                                   dataset['thetas'], 
                                                                   compute_precision(dataset['data'],
                                                                                     st,
                                                                                     spectral.labels_))
            except:
                print("Spectral failed")


            
            try :
                kmeans.fit(dataset['data'])
                res_all[(st, dim, mm, 'kmeans')] = results_recap(dataset['states'], 
                                                                  kmeans.labels_,
                                                                  dataset['thetas'], 
                                                                  compute_precision(dataset['data'],
                                                                                    st,
                                                                                    kmeans.labels_))
            except:
                print("Kmeans failed")

                

with open('Cluster_methods_comparison.pickle', 'wb') as f:
    pkl.dump([res_all], f)

  0%|          | 0/1 [00:00<?, ?it/s]

Case: N_states: 2 ,TS dim: 10 ,Prec mode: regain



  0%|          | 0/1 [00:00<?, ?it/s]

Case: N_states: 2 ,TS dim: 10 ,Prec mode: regain_random



  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective d

Case: N_states: 15 ,TS dim: 30 ,Prec mode: regain


  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/hmm/hmm_graphical_lasso.py:257: UserWarning: The optimisation did not converge.
  warnings.warn('The optimisation did not converge.')

100%|██████████| 20/20 [3:16:03<00:00, 588.17s/it]


Case: N_states: 15 ,TS dim: 30 ,Prec mode: regain_random


  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective did not converge.
  warnings.warn("Objective did not converge.")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/federicociech/PycharmProjects/regain/regain/covariance/graphical_lasso_.py:192: UserWarning: Objective d

In [5]:
res_all

{(2, 10, 'regain', 'HMM_GMM'): {'homogeneity [0, 1]': 0.9367975327785221,
  'completeness [0, 1]': 0.9354921245942696,
  'v_measure [0, 1]': 0.9361443736044269,
  'adjusted_mutual_info [0, 1]': 0.9360979920074415,
  'weighted_mean_mcc [-1, 1]': (0.8553518092952413,),
  'max_cluster_mean_mcc[-1,1]': (0.8625165649027264,),
  'weighted_mean_f1 [0, 1]': (0.8130997676610452,),
  'max_cluster_mean_f1[0,1]': (0.8166666666666667,),
  'probabilities_clusters': (array([[0.01511879, 0.9981378 ],
          [0.98488121, 0.0018622 ]]),),
  'max_probabilities_couples': ['Couple: (0, 1), Probability: 0.9981378026070763, MCC: 0.8386278693775346, F1_score: 0.8333333333333333',
   'Couple: (1, 0), Probability: 0.9848812095032398, MCC: 0.8864052604279183, F1_score: 0.8']},
 (2, 10, 'regain', 'GMM'): {'homogeneity [0, 1]': 0.9535325112599524,
  'completeness [0, 1]': 0.9522037832257766,
  'v_measure [0, 1]': 0.952867684031298,
  'adjusted_mutual_info [0, 1]': 0.9528334494272757,
  'weighted_mean_mcc [-1, 1

In [1]:
import pickle

with open('Cluster_methods_comparison.pickle', 'rb') as f:
   [trial] =  pickle.load(f)

In [18]:
trial[(2, 10, 'regain', 'HMM_GMM')]['max_probabilities_couples'][1][12]

'0'